# Generating ML XS, ACE Files, Benchmark Inputs and Serpent Scripts

In [1]:
import pandas as pd
import os
import logging

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
pd.options.mode.chained_assignment = None  # default='warn'

import sys
# This allows us to import the nucml utilities
sys.path.append("..")


import nucml.datasets as nuc_data
import nucml.ace.data_utilities as ace_utils
import nucml.model.utilities as model_utils

In [2]:
figure_dir = "Figures/"

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [4]:
import importlib
importlib.reload(nuc_data)
importlib.reload(ace_utils)
importlib.reload(model_utils)
print("Finish re-loading scripts.")

Finish re-loading scripts.


In [5]:
import nucml.exfor.data_utilities as exfor_utils
import nucml.general_utilities as gen_utils
import nucml.config as config
importlib.reload(config)
importlib.reload(gen_utils)
importlib.reload(exfor_utils)

<module 'nucml.exfor.data_utilities' from '..\\nucml\\exfor\\data_utilities.py'>

## B0 Dataset

In [6]:
# LOADING DATASET
df_b0, _, _, _, _, to_scale_b0, _ = nuc_data.load_exfor(pedro=True, basic=0)

### Decision Tree

In [15]:
dt_ml_ace_dir = "ml/DT_B0/"

In [16]:
dt_results_filepath = "../ML_EXFOR_neutrons/2_DT/dt_resultsB0.csv"
dt_results = pd.read_csv(dt_results_filepath)

# filter down to just three for testing purposes:
# dt_results = model_utils.get_best_models_df(dt_results)

dt_results

,id,max_depth,mss,msl,mt_strategy,normalizer,train_mae,train_mse,train_evs,train_mae_m,train_r2,val_mae,val_mse,val_evs,val_mae_m,val_r2,test_mae,test_mse,test_evs,test_mae_m,test_r2,model_path,training_time,scaler_path
0,1,60,2,1,one_hot,standard,0.027246,0.011597,0.984938,0.000000,0.984938,0.134148,0.079009,0.898342,0.047617,0.898340,0.135704,0.081381,0.895014,0.047966,0.895013,E:\ML_Models_EXFOR\DT_B0\DT60_MSS2_MSL1_standa...,37.584735,E:\ML_Models_EXFOR\DT_B0\DT60_MSS2_MSL1_standa...
1,6,60,5,1,one_hot,standard,0.053034,0.017223,0.977466,0.014151,0.977466,0.129590,0.072954,0.905400,0.046617,0.905398,0.130816,0.074608,0.902892,0.046723,0.902892,E:\ML_Models_EXFOR\DT_B0\DT60_MSS5_MSL1_standa...,35.939270,E:\ML_Models_EXFOR\DT_B0\DT60_MSS5_MSL1_standa...
2,7,60,5,3,one_hot,standard,0.077898,0.028393,0.962302,0.026635,0.962302,0.122352,0.061606,0.918759,0.045988,0.918759,0.123353,0.062753,0.916827,0.045903,0.916826,E:\ML_Models_EXFOR\DT_B0\DT60_MSS5_MSL3_standa...,34.894254,E:\ML_Models_EXFOR\DT_B0\DT60_MSS5_MSL3_standa...
3,11,60,10,1,one_hot,standard,0.070885,0.024126,0.968148,0.024299,0.968148,0.124314,0.066369,0.913028,0.046032,0.913026,0.125661,0.068029,0.910458,0.045964,0.910458,E:\ML_Models_EXFOR\DT_B0\DT60_MSS10_MSL1_stand...,35.069544,E:\ML_Models_EXFOR\DT_B0\DT60_MSS10_MSL1_stand...
4,12,60,10,3,one_hot,standard,0.082616,0.030622,0.959222,0.029597,0.959222,0.121269,0.059992,0.920611,0.046077,0.920611,0.122192,0.061186,0.918584,0.046094,0.918583,E:\ML_Models_EXFOR\DT_B0\DT60_MSS10_MSL3_stand...,35.305323,E:\ML_Models_EXFOR\DT_B0\DT60_MSS10_MSL3_stand...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,494,84,10,7,one_hot,none,0.094411,0.037966,0.948927,0.035953,0.948927,0.119692,0.057610,0.923213,0.046837,0.923212,0.119076,0.057716,0.922230,0.046595,0.922230,E:\ML_Models_EXFOR\DT_B0\DT84_MSS10_MSL7_none_...,37.965002,E:\ML_Models_EXFOR\DT_B0\DT84_MSS10_MSL7_none_...
377,497,170,15,3,one_hot,none,0.088061,0.033275,0.955518,0.032893,0.955518,0.121129,0.059571,0.921086,0.046407,0.921085,0.121108,0.060124,0.919441,0.046460,0.919441,E:\ML_Models_EXFOR\DT_B0\DT170_MSS15_MSL3_none...,38.199000,E:\ML_Models_EXFOR\DT_B0\DT170_MSS15_MSL3_none...
378,498,126,15,5,one_hot,none,0.091553,0.035765,0.952030,0.034612,0.952030,0.120049,0.058212,0.922746,0.046717,0.922746,0.119821,0.058480,0.921370,0.046692,0.921370,E:\ML_Models_EXFOR\DT_B0\DT126_MSS15_MSL5_none...,38.689017,E:\ML_Models_EXFOR\DT_B0\DT126_MSS15_MSL5_none...
379,499,84,15,7,one_hot,none,0.094736,0.038127,0.948700,0.036156,0.948700,0.119781,0.057766,0.922990,0.046935,0.922989,0.119134,0.057673,0.922256,0.046699,0.922255,E:\ML_Models_EXFOR\DT_B0\DT84_MSS15_MSL7_none_...,38.435998,E:\ML_Models_EXFOR\DT_B0\DT84_MSS15_MSL7_none_...


In [17]:
BENCHMARK_NAME = "U233-MET-FAST-001"
ace_utils.generate_bench_ml_xs(df_b0, dt_results, BENCHMARK_NAME, to_scale_b0, dt_ml_ace_dir, reset=True)

In [18]:
ace_utils.generate_serpent_bash(dt_ml_ace_dir, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

In [11]:
# all_serpent_files = []

# for root, _, files in os.walk("ml/DT_B0"):
#     for file in files:
#         if "U233_MET_FAST_001_001" in root:
#             all_serpent_files.append(os.path.abspath(os.path.join(root, file)))

### K-Nearest-Neighbors

In [25]:
knn_ml_ace_dir = "ml/KNN_B0/"

In [26]:
knn_results_filepath = "../ML_EXFOR_neutrons/1_KNN/knn_results_B0.csv"
knn_results = pd.read_csv(knn_results_filepath)
knn_results["scale_energy"] = knn_results.run_name.apply(lambda x: True if "v2" in x else False)
# knn_results = model_utils.get_best_models_df(knn_results)
knn_results

,id,distance_metric,mt_strategy,normalizer,train_mae,train_mse,train_evs,train_mae_m,train_r2,val_mae,val_mse,val_evs,val_mae_m,val_r2,test_mae,test_mse,test_evs,test_mae_m,test_r2,model_path,training_time,scaler_path,run_name,scale_energy
0,13,euclidean,one_hot,standard,0.025880,0.010936,0.985814,0.0,0.985814,0.118292,0.062828,0.915687,0.042361,0.915686,0.118816,0.063698,0.914577,0.042778,0.914573,E:\ML_Models_EXFOR\KNN_B0\k13_distance_euclide...,3671.971000,E:\ML_Models_EXFOR\KNN_B0\k13_distance_euclide...,k13_distance_euclidean_standard_one_hot_B0_v1,False
1,9,euclidean,one_hot,standard,0.025931,0.010999,0.985734,0.0,0.985734,0.118136,0.062010,0.917756,0.041772,0.917756,0.118899,0.064408,0.913698,0.041733,0.913698,E:\ML_Models_EXFOR\KNN_B0\k9_distance_euclidea...,4992.854609,E:\ML_Models_EXFOR\KNN_B0\k9_distance_euclidea...,k9_distance_euclidean_standard_one_hot_B0_v1,False
2,1,euclidean,one_hot,standard,0.029060,0.021444,0.972555,0.0,0.972555,0.140604,0.090440,0.885060,0.047921,0.885059,0.140549,0.091576,0.882881,0.047418,0.882880,E:\ML_Models_EXFOR\KNN_B0\k1_distance_euclidea...,6258.326797,E:\ML_Models_EXFOR\KNN_B0\k1_distance_euclidea...,k1_distance_euclidean_standard_one_hot_B0_v1,False
3,5,euclidean,one_hot,standard,0.026062,0.011286,0.985346,0.0,0.985346,0.119661,0.066503,0.911902,0.041176,0.911900,0.119958,0.067081,0.911042,0.041299,0.911040,E:\ML_Models_EXFOR\KNN_B0\k5_distance_euclidea...,6381.527638,E:\ML_Models_EXFOR\KNN_B0\k5_distance_euclidea...,k5_distance_euclidean_standard_one_hot_B0_v1,False
4,17,euclidean,one_hot,standard,0.025844,0.010874,0.985899,0.0,0.985899,0.119776,0.064781,0.912524,0.043461,0.912522,0.119740,0.064122,0.913774,0.043261,0.913773,E:\ML_Models_EXFOR\KNN_B0\k17_distance_euclide...,6478.077649,E:\ML_Models_EXFOR\KNN_B0\k17_distance_euclide...,k17_distance_euclidean_standard_one_hot_B0_v1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,4,euclidean,one_hot,minmax,0.026206,0.011535,0.985050,0.0,0.985050,0.119090,0.063827,0.916232,0.040870,0.916232,0.118699,0.064212,0.914864,0.040746,0.914864,E:\ML_Models_EXFOR\KNN_B0\k4_distance_euclidea...,52908.405799,E:\ML_Models_EXFOR\KNN_B0\k4_distance_euclidea...,k4_distance_euclidean_minmax_one_hot_B0_v2,True
276,8,manhattan,one_hot,minmax,0.025926,0.011033,0.985687,0.0,0.985687,0.116317,0.060090,0.920073,0.041105,0.920073,0.115661,0.058934,0.921851,0.040820,0.921849,E:\ML_Models_EXFOR\KNN_B0\k8_distance_manhatta...,45342.842937,E:\ML_Models_EXFOR\KNN_B0\k8_distance_manhatta...,k8_distance_manhattan_minmax_one_hot_B0_v2,True
277,12,manhattan,one_hot,minmax,0.025839,0.010890,0.985876,0.0,0.985876,0.115830,0.059749,0.920521,0.041403,0.920521,0.115695,0.059854,0.920381,0.041307,0.920381,E:\ML_Models_EXFOR\KNN_B0\k12_distance_manhatt...,48068.513939,E:\ML_Models_EXFOR\KNN_B0\k12_distance_manhatt...,k12_distance_manhattan_minmax_one_hot_B0_v2,True
278,16,manhattan,one_hot,minmax,0.025825,0.010862,0.985911,0.0,0.985911,0.115843,0.059055,0.921012,0.041857,0.921012,0.116128,0.059337,0.921421,0.042361,0.921421,E:\ML_Models_EXFOR\KNN_B0\k16_distance_manhatt...,46104.734946,E:\ML_Models_EXFOR\KNN_B0\k16_distance_manhatt...,k16_distance_manhattan_minmax_one_hot_B0_v2,True


In [27]:
ace_utils.generate_bench_ml_xs(df_b0, knn_results, "U233-MET-FAST-001", to_scale_b0, knn_ml_ace_dir, reset=True)

In [84]:
ace_utils.generate_serpent_bash("ml/KNN_B0/")